In [2]:
import numpy as np
import json
from sentence_transformers import SentenceTransformer, util

In [4]:
DATA_FILE = 'dummy_chatbot_data.json'

with open(DATA_FILE, 'r') as f:
    data = json.load(f)

In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [26]:
all_questions = []
question_to_intent_map = []

for intent_data in data:
    intent_name = intent_data['intent']
    answer_text = intent_data['answer']
    for question_text in intent_data['questions']:
        all_questions.append(question_text)
        question_to_intent_map.append({
            "intent": intent_name,
            "question_text": question_text, # Storing for debugging/verification
            "answer": answer_text
        })
question_embeddings = model.encode(all_questions, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
def get_chatbot_response(user_query, similarity_threshold=0.7):
    if not user_query.strip():
        return 'Please ask me a question!'
    
    query_embedding = model.encode(user_query, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(query_embedding, question_embeddings)[0]
    
    most_similar_idx = np.argmax(cosine_scores.cpu().numpy())
    best_score = cosine_scores[most_similar_idx].item()
    
    print(f"\nUser: {user_query}")

    if best_score >= similarity_threshold:
        # If a good match is found, return the answer associated with that intent
        matched_data = question_to_intent_map[most_similar_idx]
        return matched_data['answer']
    else:
        # If no good match, provide a fallback
        return "I'm sorry, I don't quite understand your question. Could you please rephrase it or ask something different?"


In [27]:
if __name__ == "__main__":
    print("\nGradLink AI Chat Assistant (Type 'quit' to exit)")
    print("-" * 40)

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            print("Chatbot: Goodbye! Have a great day.")
            break
        
        response = get_chatbot_response(user_input)
        print(f"Chatbot: {response}")
        print("-" * 40) # Separator for clarity


GradLink AI Chat Assistant (Type 'quit' to exit)
----------------------------------------

User: how do I get a schorlaship
Chatbot: I'm sorry, I don't quite understand your question. Could you please rephrase it or ask something different?
----------------------------------------

User: how do I get scholarship
Chatbot: To find scholarships and funding, explore your university's scholarship board, check government scholarship schemes (e.g., PTDF, NNPC), visit reputable scholarship websites (e.g., Scholars4Dev, Opportunity Desk), and look into corporate sponsorships. Networking and excellent academic performance are key.
----------------------------------------
Chatbot: Goodbye! Have a great day.
